# Extraction

In [101]:
import pandas as pd
active_corporations=pd.read_csv(r'Active_Corporations___Beginning_1800.csv')
unemployment=pd.read_csv(r'Local_Area_Unemployment_Statistics__Beginning_1976.csv')
counties=pd.read_csv(r'counties.csv')

In [10]:
counties.head()

,County,FIPS Code[1],County Seat[2],Created[2],Formed from[3],Named for[4],Density,2010 Population[2],Area[2],Map
0,Albany County,1,Albany,1683,One of 12 original counties created in the New...,James II of England (James VII of Scotland) (1...,570.74,"304,204","533 sq mi(1,380 km2)",NaN
1,Allegany County,3,Belmont,1806,Genesee County,A variant spelling of the Allegheny River,47.34,"48,946","1,034 sq mi(2,678 km2)",NaN
2,Bronx County,5,New York City (coextensive withThe Bronx),1914[5],New York County,"Jonas Bronck (1600?–1643), an early settler of...","24,118.2","1,385,108",57.43 sq mi(149 km2),NaN
3,Broome County,7,Binghamton,1806,Tioga County,"John Broome (1738–1810), fourth Lieutenant Gov...",280.56,"200,600","715 sq mi(1,852 km2)",NaN
4,Cattaraugus County,9,Little Valley,1808,Genesee County,"A Seneca word meaning ""bad smelling banks"", re...",61.31,"80,317","1,310 sq mi(3,393 km2)",NaN


In [11]:
unemployment.head()

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate
0,Albany City,2021,2,46700,43000,3700,7.9
1,Albany City,2021,1,46800,43100,3700,7.9
2,Albany City,2020,12,47200,43500,3700,7.8
3,Albany City,2020,11,47400,43800,3600,7.7
4,Albany City,2020,10,47200,43500,3800,8.0


In [13]:
active_corporations.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent Address 2,Registered Agent City,Registered Agent State,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip
0,3211809,00:02:59 LLC,05/31/2005,KINGS,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,00:02:59 LLC,"656 UNION STREET, APT. 2",NaN,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4472700,000 LLC,10/15/2013,KINGS,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,000,304 MESEROLE ST #2A,NaN,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4723823,"000 MEADOW LANE, LLC",03/11/2015,SUFFOLK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O ACKERMAN O'BRIEN PACHMAN & BROWN LLP,34 PANTIGO ROAD,NaN,EAST HAMPTON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5224240,00168 HOME CORP.,10/26/2017,SUFFOLK,NEW YORK,DOMESTIC BUSINESS CORPORATION,00168 HOME CORP.,28 PHEASANT RUN LANE,NaN,DIX HILLS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5581065,"0016 HEMPSTEAD TURNPIKE, LLC",07/02/2019,SUFFOLK,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"GOLDENBOCK, EISEMAN, ASSOR, BELL & PESKOE LLP",ATTN LARRY A WELCH ESQ,711 3RD AVE FL 16,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Transform

## Data Cleaning

In [102]:
not_null_date=active_corporations['Initial DOS Filing Date'].isnull().map(lambda x: not x)
active_corporations=active_corporations[not_null_date]
active_corporations['month']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[0]))
active_corporations['day']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[1]))
active_corporations['year']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[2]))
active_corporations=active_corporations[active_corporations['year'].map(lambda x: x<1976)]

In [107]:
def format_county(s):
    suffix=' County'
    if s.endswith(suffix):
        return s[:-len(suffix)].lower()
    return s.lower()
county_names=counties['County']
county_names=set(map(format_county,county_names))

In [115]:
not_null_county=active_corporations['County'].isnull().map(lambda x: not x)
active_corporations=active_corporations[not_null_county]
active_corporations['County']=active_corporations['County'].map(lambda x: x.lower())
active_corporations=active_corporations[active_corporations['County'].map(lambda x: x in county_names)]